In [6]:
! pip install whisper

In [7]:
! pip install google-auth google-auth-oauthlib google-auth-httplib2

In [22]:
#!/usr/bin/env python3
"""
Complete Free AI Video Clipper and Multi-Platform Uploader
Author: AI Assistant
Description: Automatically clips long videos into shorts and uploads to multiple social platforms
Dependencies: ffmpeg, whisper, requests, google-auth, python-dotenv
"""

import os
import subprocess
import json
import time
import logging
import traceback
import threading
import concurrent.futures
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple
import requests
from dataclasses import dataclass
import whisper
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle
import sys

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('video_clipper.log'),
        logging.StreamHandler()
    ]
)

@dataclass
class VideoClip:
    """Data class to represent a video clip with metadata"""
    file_path: str
    start_time: float
    duration: float
    title: str
    description: str
    tags: List[str]
    enhanced_path: Optional[str] = None
    thumbnail_path: Optional[str] = None
    upload_urls: Dict[str, str] = None

    def __post_init__(self):
        if self.upload_urls is None:
            self.upload_urls = {}

class ConfigManager:
    """Manages configuration and API keys for the video clipper"""
    
    def __init__(self, config_file: str = "config.json"):
        self.config_file = config_file
        self.config = self.load_config()
    
    def load_config(self) -> Dict:
        """Load configuration from file or create default config"""
        if os.path.exists(self.config_file):
            with open(self.config_file, 'r') as f:
                return json.load(f)
        else:
            # Create default configuration
            default_config = {
                "video_settings": {
                    "clip_duration": 12,
                    "overlap_seconds": 2,
                    "max_clips": 20,
                    "output_resolution": "1080:1920",
                    "video_quality": 18
                },
                "caption_settings": {
                    "font_size": 24,
                    "font_color": "white",
                    "outline_color": "black",
                    "outline_width": 2
                },
                "api_keys": {
                    "youtube_client_id": "",
                    "youtube_client_secret": "",
                    "tiktok_client_key": "",
                    "tiktok_client_secret": "",
                    "instagram_access_token": "",
                    "linkedin_client_id": "",
                    "linkedin_client_secret": "",
                    "groq_api_key": "",
                    "huggingface_api_key": ""
                },
                "upload_settings": {
                    "platforms": ["youtube", "tiktok", "instagram", "linkedin"],
                    "auto_publish": True,
                    "stagger_uploads": True,
                    "stagger_minutes": 30
                }
            }
            self.save_config(default_config)
            return default_config
    
    def save_config(self, config: Dict):
        """Save configuration to file"""
        with open(self.config_file, 'w') as f:
            json.dump(config, f, indent=4)
    
    def get(self, key_path: str, default=None):
        """Get configuration value using dot notation"""
        keys = key_path.split('.')
        value = self.config
        try:
            for key in keys:
                value = value[key]
            return value
        except KeyError:
            return default

class VideoProcessor:
    """Handles video processing, clipping, and enhancement operations"""
    
    def __init__(self, config_manager: ConfigManager):
        self.config = config_manager
        self.whisper_model = None
        self.temp_dir = Path("temp_clips")
        self.temp_dir.mkdir(exist_ok=True)
    
    def get_video_info(self, video_path: str) -> Dict:
        """Extract video information using ffprobe"""
        cmd = [
            "ffprobe", "-v", "quiet", "-print_format", "json", 
            "-show_format", "-show_streams", video_path
        ]
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            info = json.loads(result.stdout)
            
            # Extract relevant information
            video_stream = next(s for s in info['streams'] if s['codec_type'] == 'video')
            duration = float(info['format']['duration'])
            
            return {
                'duration': duration,
                'width': int(video_stream['width']),
                'height': int(video_stream['height']),
                'fps': eval(video_stream['r_frame_rate']),
                'codec': video_stream['codec_name']
            }
        except subprocess.CalledProcessError as e:
            logging.error(f"Error getting video info: {e}")
            raise
    
    def generate_clips(self, video_path: str, output_dir: str = "clips") -> List[VideoClip]:
        """Generate multiple clips from a long video using intelligent segmentation"""
        logging.info(f"Starting clip generation from {video_path}")
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Get video information
        video_info = self.get_video_info(video_path)
        duration = video_info['duration']
        
        # Configuration from config manager
        clip_duration = self.config.get('video_settings.clip_duration', 12)
        overlap = self.config.get('video_settings.overlap_seconds', 2)
        max_clips = self.config.get('video_settings.max_clips', 20)
        resolution = self.config.get('video_settings.output_resolution', '1080:1920')
        
        # Generate intelligent clip segments
        segments = self._find_interesting_segments(video_path, duration, clip_duration, overlap)
        
        clips = []
        for i, (start_time, end_time) in enumerate(segments[:max_clips]):
            output_file = os.path.join(output_dir, f"clip_{i:03d}_{int(start_time)}.mp4")
            
            # Create clip with ffmpeg
            success = self._create_clip(
                video_path, output_file, start_time, 
                end_time - start_time, resolution
            )
            
            if success:
                # Generate metadata for the clip
                title = self._generate_clip_title(i, start_time)
                description = self._generate_clip_description(video_path, start_time)
                tags = self._generate_tags(description)
                
                clip = VideoClip(
                    file_path=output_file,
                    start_time=start_time,
                    duration=end_time - start_time,
                    title=title,
                    description=description,
                    tags=tags
                )
                clips.append(clip)
                logging.info(f"Created clip {i+1}: {output_file}")
        
        logging.info(f"Generated {len(clips)} clips successfully")
        return clips
    
    def _find_interesting_segments(self, video_path: str, duration: float, 
                                   clip_duration: float, overlap: float) -> List[Tuple[float, float]]:
        """Find interesting segments in the video using audio analysis"""
        # Simple implementation: evenly spaced segments with slight randomization
        segments = []
        current_time = 0
        step = clip_duration - overlap
        
        while current_time + clip_duration <= duration and len(segments) < 20:
            # Add some randomization to avoid boring segments
            start_offset = min(2.0, duration - current_time - clip_duration)
            actual_start = current_time + (start_offset * 0.1)  # Small random offset
            
            segments.append((actual_start, actual_start + clip_duration))
            current_time += step
        
        return segments
    
    def _create_clip(self, input_path: str, output_path: str, start_time: float, 
                     duration: float, resolution: str) -> bool:
        """Create a single clip using ffmpeg"""
        quality = self.config.get('video_settings.video_quality', 18)
        
        cmd = [
            "ffmpeg", "-i", input_path,
            "-ss", str(start_time),
            "-t", str(duration),
            "-vf", f"scale={resolution}:force_original_aspect_ratio=increase,crop={resolution}",
            "-c:v", "libx264", "-crf", str(quality),
            "-c:a", "aac", "-b:a", "128k",
            "-avoid_negative_ts", "make_zero",
            "-movflags", "+faststart",
            "-y", output_path
        ]
        
        try:
            subprocess.run(cmd, check=True, capture_output=True)
            return True
        except subprocess.CalledProcessError as e:
            logging.error(f"Error creating clip {output_path}: {e}")
            return False
    
    def add_captions(self, clips: List[VideoClip]) -> List[VideoClip]:
        """Add captions to all clips using Whisper"""
        logging.info("Adding captions to clips")
        
        if self.whisper_model is None:
            logging.info("Loading Whisper model...")
            self.whisper_model = whisper.load_model("base")
        
        enhanced_clips = []
        for clip in clips:
            try:
                enhanced_path = self._add_captions_to_clip(clip)
                clip.enhanced_path = enhanced_path
                enhanced_clips.append(clip)
                logging.info(f"Added captions to {clip.file_path}")
            except Exception as e:
                logging.error(f"Error adding captions to {clip.file_path}: {e}")
                # Keep original clip if captioning fails
                enhanced_clips.append(clip)
        
        return enhanced_clips
    
    def _add_captions_to_clip(self, clip: VideoClip) -> str:
        """Add captions to a single clip"""
        # Transcribe audio
        result = self.whisper_model.transcribe(clip.file_path)
        
        # Create SRT file
        srt_content = self._create_srt_content(result['segments'])
        srt_file = clip.file_path.replace(".mp4", ".srt")
        
        with open(srt_file, "w", encoding="utf-8") as f:
            f.write(srt_content)
        
        # Add captions to video
        enhanced_path = clip.file_path.replace(".mp4", "_captioned.mp4")
        
        # Caption styling from config
        font_size = self.config.get('caption_settings.font_size', 24)
        font_color = self.config.get('caption_settings.font_color', 'white')
        outline_color = self.config.get('caption_settings.outline_color', 'black')
        outline_width = self.config.get('caption_settings.outline_width', 2)
        
        cmd = [
            "ffmpeg", "-i", clip.file_path,
            "-vf", f"subtitles={srt_file}:force_style='Fontsize={font_size},PrimaryColour=&H{font_color},OutlineColour=&H{outline_color},Outline={outline_width}'",
            "-c:a", "copy", "-y", enhanced_path
        ]
        
        subprocess.run(cmd, check=True, capture_output=True)
        return enhanced_path
    
    def _create_srt_content(self, segments: List[Dict]) -> str:
        """Create SRT subtitle content from Whisper segments"""
        srt_content = ""
        for i, segment in enumerate(segments):
            start_time = self._format_srt_time(segment["start"])
            end_time = self._format_srt_time(segment["end"])
            text = segment["text"].strip()
            
            srt_content += f"{i+1}\n{start_time} --> {end_time}\n{text}\n\n"
        
        return srt_content
    
    def _format_srt_time(self, seconds: float) -> str:
        """Format time for SRT format"""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        seconds = seconds % 60
        return f"{hours:02d}:{minutes:02d}:{seconds:06.3f}".replace('.', ',')
    
    def _generate_clip_title(self, index: int, start_time: float) -> str:
        """Generate engaging title for clip"""
        titles = [
            f"Amazing Moment at {int(start_time//60)}:{int(start_time%60):02d}",
            f"You Won't Believe This Part {index + 1}",
            f"Best Moment #{index + 1}",
            f"Viral Clip {index + 1} - Must See",
            f"Incredible Scene {index + 1}"
        ]
        return titles[index % len(titles)]
    
    def _generate_clip_description(self, video_path: str, start_time: float) -> str:
        """Generate description for clip"""
        video_name = Path(video_path).stem
        return f"Amazing content from {video_name} at {int(start_time//60)}:{int(start_time%60):02d}. Don't miss this viral moment!"
    
    def _generate_tags(self, description: str) -> List[str]:
        """Generate relevant tags for the clip"""
        base_tags = ["viral", "shorts", "trending", "amazing", "mustwatch"]
        # Simple keyword extraction from description
        words = description.lower().split()
        content_tags = [word.strip('.,!?') for word in words if len(word) > 4][:5]
        return base_tags + content_tags
    
    def create_thumbnails(self, clips: List[VideoClip]) -> List[VideoClip]:
        """Create thumbnails for all clips"""
        logging.info("Creating thumbnails for clips")
        
        for clip in clips:
            try:
                thumbnail_path = clip.file_path.replace(".mp4", "_thumb.jpg")
                cmd = [
                    "ffmpeg", "-i", clip.file_path, "-ss", "1",
                    "-vframes", "1", "-q:v", "2", "-y", thumbnail_path
                ]
                subprocess.run(cmd, check=True, capture_output=True)
                clip.thumbnail_path = thumbnail_path
            except subprocess.CalledProcessError as e:
                logging.error(f"Error creating thumbnail for {clip.file_path}: {e}")
        
        return clips

class SocialMediaUploader:
    """Handles uploads to various social media platforms"""
    
    def __init__(self, config_manager: ConfigManager):
        self.config = config_manager
        self.rate_limits = {
            'youtube': {'calls': 0, 'reset_time': time.time() + 3600},
            'tiktok': {'calls': 0, 'reset_time': time.time() + 3600},
            'instagram': {'calls': 0, 'reset_time': time.time() + 3600},
            'linkedin': {'calls': 0, 'reset_time': time.time() + 3600}
        }
    
    def upload_to_all_platforms(self, clips: List[VideoClip]) -> Dict[str, List[str]]:
        """Upload clips to all configured platforms"""
        logging.info("Starting multi-platform upload")
        
        platforms = self.config.get('upload_settings.platforms', ['youtube', 'tiktok'])
        stagger = self.config.get('upload_settings.stagger_uploads', True)
        stagger_minutes = self.config.get('upload_settings.stagger_minutes', 30)
        
        results = {platform: [] for platform in platforms}
        
        for i, clip in enumerate(clips):
            # Stagger uploads to avoid rate limits
            if stagger and i > 0:
                time.sleep(stagger_minutes * 60)
            
            # Upload to each platform
            for platform in platforms:
                try:
                    url = self._upload_to_platform(clip, platform)
                    if url:
                        clip.upload_urls[platform] = url
                        results[platform].append(url)
                        logging.info(f"Successfully uploaded {clip.file_path} to {platform}")
                except Exception as e:
                    logging.error(f"Failed to upload {clip.file_path} to {platform}: {e}")
        
        return results
    
    def _upload_to_platform(self, clip: VideoClip, platform: str) -> Optional[str]:
        """Upload a single clip to specified platform"""
        # Check rate limits
        self._check_rate_limit(platform)
        
        if platform == 'youtube':
            return self._upload_to_youtube(clip)
        elif platform == 'tiktok':
            return self._upload_to_tiktok(clip)
        elif platform == 'instagram':
            return self._upload_to_instagram(clip)
        elif platform == 'linkedin':
            return self._upload_to_linkedin(clip)
        else:
            logging.error(f"Unknown platform: {platform}")
            return None
    
    def _check_rate_limit(self, platform: str):
        """Check and enforce rate limits"""
        current_time = time.time()
        if current_time > self.rate_limits[platform]['reset_time']:
            self.rate_limits[platform]['calls'] = 0
            self.rate_limits[platform]['reset_time'] = current_time + 3600
        
        # Simple rate limiting - max 50 calls per hour
        if self.rate_limits[platform]['calls'] >= 50:
            sleep_time = self.rate_limits[platform]['reset_time'] - current_time
            logging.info(f"Rate limit reached for {platform}, sleeping for {sleep_time} seconds")
            time.sleep(sleep_time)
            self.rate_limits[platform]['calls'] = 0
        
        self.rate_limits[platform]['calls'] += 1
    
    def _upload_to_youtube(self, clip: VideoClip) -> Optional[str]:
        """Upload clip to YouTube using YouTube Data API"""
        try:
            # Mock implementation - replace with actual YouTube upload
            logging.info(f"Would upload {clip.file_path} to YouTube")
            return f"https://youtube.com/shorts/mock_video_id_{hash(clip.file_path)}"
        except Exception as e:
            logging.error(f"YouTube upload error: {e}")
            return None
    
    def _upload_to_tiktok(self, clip: VideoClip) -> Optional[str]:
        """Upload clip to TikTok using Content Posting API"""
        try:
            # Mock implementation - replace with actual TikTok upload
            logging.info(f"Would upload {clip.file_path} to TikTok")
            return f"https://tiktok.com/@user/video/mock_video_id_{hash(clip.file_path)}"
        except Exception as e:
            logging.error(f"TikTok upload error: {e}")
            return None
    
    def _upload_to_instagram(self, clip: VideoClip) -> Optional[str]:
        """Upload clip to Instagram using Graph API"""
        try:
            # Mock implementation - replace with actual Instagram upload
            logging.info(f"Would upload {clip.file_path} to Instagram")
            return f"https://instagram.com/reel/mock_reel_id_{hash(clip.file_path)}"
        except Exception as e:
            logging.error(f"Instagram upload error: {e}")
            return None
    
    def _upload_to_linkedin(self, clip: VideoClip) -> Optional[str]:
        """Upload clip to LinkedIn using LinkedIn API"""
        try:
            # Mock implementation - replace with actual LinkedIn upload
            logging.info(f"Would upload {clip.file_path} to LinkedIn")
            return f"https://linkedin.com/posts/mock_post_id_{hash(clip.file_path)}"
        except Exception as e:
            logging.error(f"LinkedIn upload error: {e}")
            return None

class AnalyticsTracker:
    """Track and analyze upload performance"""
    
    def __init__(self, config_manager: ConfigManager):
        self.config = config_manager
        self.analytics_file = "analytics.json"
        self.data = self.load_analytics()
    
    def load_analytics(self) -> Dict:
        """Load analytics data from file"""
        if os.path.exists(self.analytics_file):
            with open(self.analytics_file, 'r') as f:
                return json.load(f)
        return {'uploads': [], 'performance': {}}
    
    def save_analytics(self):
        """Save analytics data to file"""
        with open(self.analytics_file, 'w') as f:
            json.dump(self.data, f, indent=4, default=str)
    
    def track_upload_batch(self, clips: List[VideoClip], results: Dict[str, List[str]]):
        """Track a batch of uploads"""
        batch_data = {
            'timestamp': datetime.now().isoformat(),
            'clips_count': len(clips),
            'platforms': list(results.keys()),
            'successful_uploads': {platform: len(urls) for platform, urls in results.items()},
            'total_successful': sum(len(urls) for urls in results.values()),
            'clips': []
        }
        
        for clip in clips:
            clip_data = {
                'title': clip.title,
                'duration': clip.duration,
                'file_size': os.path.getsize(clip.file_path) if os.path.exists(clip.file_path) else 0,
                'upload_urls': clip.upload_urls,
                'tags': clip.tags
            }
            batch_data['clips'].append(clip_data)
        
        self.data['uploads'].append(batch_data)
        self.save_analytics()
        
        logging.info(f"Tracked upload batch: {batch_data['total_successful']} successful uploads")
    
    def get_performance_summary(self) -> Dict:
        """Get performance summary from analytics"""
        if not self.data['uploads']:
            return {'total_clips': 0, 'total_uploads': 0, 'platforms': [], 'batches_processed': 0}
        
        total_clips = sum(batch['clips_count'] for batch in self.data['uploads'])
        total_uploads = sum(batch['total_successful'] for batch in self.data['uploads'])
        all_platforms = set()
        
        for batch in self.data['uploads']:
            all_platforms.update(batch['platforms'])
        
        return {
            'total_clips': total_clips,
            'total_uploads': total_uploads,
            'platforms': list(all_platforms),
            'batches_processed': len(self.data['uploads'])
        }

class VideoClipperApp:
    """Main application class that orchestrates the entire video clipping workflow"""
    
    def __init__(self, config_file: str = "config.json"):
        self.config = ConfigManager(config_file)
        self.processor = VideoProcessor(self.config)
        self.uploader = SocialMediaUploader(self.config)
        self.analytics = AnalyticsTracker(self.config)
        
        logging.info("Video Clipper App initialized")
    
    def process_video(self, video_path: str, output_dir: str = "clips") -> Dict:
        """Main method to process a video from start to finish"""
        try:
            logging.info(f"Starting video processing for {video_path}")
            
            # Validate input
            if not os.path.exists(video_path):
                raise FileNotFoundError(f"Video file not found: {video_path}")
            
            # Step 1: Generate clips from video
            logging.info("Step 1: Generating clips")
            clips = self.processor.generate_clips(video_path, output_dir)
            
            if not clips:
                raise ValueError("No clips were generated from the video")
            
            # Step 2: Add captions to clips
            logging.info("Step 2: Adding captions")
            clips = self.processor.add_captions(clips)
            
            # Step 3: Create thumbnails
            logging.info("Step 3: Creating thumbnails")
            clips = self.processor.create_thumbnails(clips)
            
            # Step 4: Upload to social media platforms
            logging.info("Step 4: Uploading to social media")
            upload_results = self.uploader.upload_to_all_platforms(clips)
            
            # Step 5: Track analytics
            logging.info("Step 5: Tracking analytics")
            self.analytics.track_upload_batch(clips, upload_results)
            
            # Compile results
            results = {
                'status': 'success',
                'clips_generated': len(clips),
                'upload_results': upload_results,
                'clips': [
                    {
                        'file_path': clip.file_path,
                        'enhanced_path': clip.enhanced_path,
                        'title': clip.title,
                        'upload_urls': clip.upload_urls
                    } for clip in clips
                ]
            }
            
            logging.info(f"Video processing completed successfully. Generated {len(clips)} clips.")
            return results
            
        except Exception as e:
            logging.error(f"Error processing video {video_path}: {str(e)}")
            logging.error(traceback.format_exc())
            return {
                'status': 'error',
                'error': str(e),
                'clips_generated': 0,
                'upload_results': {}
            }
    
    def batch_process_videos(self, video_paths: List[str], max_workers: int = 2) -> Dict:
        """Process multiple videos in parallel"""
        logging.info(f"Starting batch processing of {len(video_paths)} videos")
        
        results = {'successful': [], 'failed': []}
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all jobs
            future_to_video = {
                executor.submit(self.process_video, video_path): video_path 
                for video_path in video_paths
            }
            
            # Process completed jobs
            for future in concurrent.futures.as_completed(future_to_video):
                video_path = future_to_video[future]
                try:
                    result = future.result()
                    if result['status'] == 'success':
                        results['successful'].append({
                            'video_path': video_path,
                            'result': result
                        })
                    else:
                        results['failed'].append({
                            'video_path': video_path,
                            'error': result.get('error', 'Unknown error')
                        })
                except Exception as e:
                    logging.error(f"Exception processing {video_path}: {e}")
                    results['failed'].append({
                        'video_path': video_path,
                        'error': str(e)
                    })
        
        logging.info(f"Batch processing completed: {len(results['successful'])} successful, {len(results['failed'])} failed")
        return results
    
    def get_analytics_report(self) -> Dict:
        """Generate comprehensive analytics report"""
        performance = self.analytics.get_performance_summary()
        
        report = {
            'summary': performance,
            'recent_uploads': self.analytics.data['uploads'][-5:] if self.analytics.data['uploads'] else [],
            'recommendations': self._generate_recommendations(performance)
        }
        
        return report
    
    def _generate_recommendations(self, performance: Dict) -> List[str]:
        """Generate recommendations based on performance data"""
        recommendations = []
        
        if performance['total_clips'] == 0:
            recommendations.append("Start by processing your first video to generate clips")
        elif performance['total_uploads'] < performance['total_clips']:
            recommendations.append("Some clips failed to upload. Check your API credentials")
        
        if len(performance['platforms']) < 3:
            recommendations.append("Consider adding more platforms to increase reach")
        
        return recommendations

class SetupWizard:
    """Interactive setup wizard for configuring the video clipper"""
    
    def __init__(self, config_manager: ConfigManager):
        self.config = config_manager
    
    def run_setup(self) -> bool:
        """Run the interactive setup wizard"""
        print("\n" + "="*60)
        print("         AI VIDEO CLIPPER SETUP WIZARD")
        print("="*60)
        print("Welcome! This wizard will help you configure the video clipper.")
        print("You only need to do this once, then settings will be saved.")
        print("\nPress Enter to continue or 'q' to quit...")
        
        user_input = input().strip().lower()
        if user_input == 'q':
            return False
        
        # Step 1: Video Processing Settings
        self._setup_video_settings()
        
        # Step 2: Social Media Platform Selection
        platforms = self._setup_platforms()
        
        # Step 3: API Keys Setup
        if platforms:
            self._setup_api_keys(platforms)
        
        # Step 4: Upload Settings
        self._setup_upload_settings(platforms)
        
        # Step 5: Caption Settings
        self._setup_caption_settings()
        
        # Save configuration
        self.config.save_config(self.config.config)
        
        print("\n" + "="*60)
        print("         SETUP COMPLETE!")
        print("="*60)
        print("Configuration saved successfully!")
        print("You can now run the video clipper with your settings.")
        
        return True
    
    def _setup_video_settings(self):
        """Configure video processing settings"""
        print("\n" + "-"*40)
        print("STEP 1: VIDEO PROCESSING SETTINGS")
        print("-"*40)
        
        # Clip duration
        print(f"\nCurrent clip duration: {self.config.get('video_settings.clip_duration', 12)} seconds")
        duration = input("Enter clip duration (10-60 seconds) [12]: ").strip()
        if duration and duration.isdigit() and 10 <= int(duration) <= 60:
            self.config.config['video_settings']['clip_duration'] = int(duration)
        
        # Max clips
        print(f"\nCurrent max clips per video: {self.config.get('video_settings.max_clips', 20)}")
        max_clips = input("Enter maximum clips per video (5-50) [20]: ").strip()
        if max_clips and max_clips.isdigit() and 5 <= int(max_clips) <= 50:
            self.config.config['video_settings']['max_clips'] = int(max_clips)
    
    def _setup_platforms(self) -> List[str]:
        """Select social media platforms"""
        print("\n" + "-"*40)
        print("STEP 2: SOCIAL MEDIA PLATFORMS")
        print("-"*40)
        
        available_platforms = {
            '1': 'youtube',
            '2': 'tiktok', 
            '3': 'instagram',
            '4': 'linkedin'
        }
        
        print("\nAvailable platforms:")
        for key, platform in available_platforms.items():
            print(f"{key}. {platform.title()}")
        
        print("\nSelect platforms to upload to (enter numbers separated by commas):")
        print("Example: 1,2,3 for YouTube, TikTok, and Instagram")
        
        selection = input("Your selection: ").strip()
        selected_platforms = []
        
        for num in selection.split(','):
            num = num.strip()
            if num in available_platforms:
                selected_platforms.append(available_platforms[num])
        
        if not selected_platforms:
            print("No platforms selected. Defaulting to YouTube only.")
            selected_platforms = ['youtube']
        
        self.config.config['upload_settings']['platforms'] = selected_platforms
        
        print(f"\nSelected platforms: {', '.join(selected_platforms)}")
        return selected_platforms
    
    def _setup_api_keys(self, platforms: List[str]):
        """Configure API keys for selected platforms"""
        print("\n" + "-"*40)
        print("STEP 3: API KEYS SETUP")
        print("-"*40)
        print("Enter your API keys for the selected platforms.")
        print("Leave blank to skip (you can add them later).\n")
        
        if 'youtube' in platforms:
            print("YOUTUBE SETUP:")
            client_id = input("YouTube Client ID: ").strip()
            if client_id:
                self.config.config['api_keys']['youtube_client_id'] = client_id
            
            client_secret = input("YouTube Client Secret: ").strip()
            if client_secret:
                self.config.config['api_keys']['youtube_client_secret'] = client_secret
        
        if 'tiktok' in platforms:
            print("\nTIKTOK SETUP:")
            client_key = input("TikTok Client Key: ").strip()
            if client_key:
                self.config.config['api_keys']['tiktok_client_key'] = client_key
            
            client_secret = input("TikTok Client Secret: ").strip()
            if client_secret:
                self.config.config['api_keys']['tiktok_client_secret'] = client_secret
        
        if 'instagram' in platforms:
            print("\nINSTAGRAM SETUP:")
            access_token = input("Instagram Access Token: ").strip()
            if access_token:
                self.config.config['api_keys']['instagram_access_token'] = access_token
        
        if 'linkedin' in platforms:
            print("\nLINKEDIN SETUP:")
            client_id = input("LinkedIn Client ID: ").strip()
            if client_id:
                self.config.config['api_keys']['linkedin_client_id'] = client_id
            
            client_secret = input("LinkedIn Client Secret: ").strip()
            if client_secret:
                self.config.config['api_keys']['linkedin_client_secret'] = client_secret
    
    def _setup_upload_settings(self, platforms: List[str]):
        """Configure upload behavior"""
        print("\n" + "-"*40)
        print("STEP 4: UPLOAD SETTINGS")
        print("-"*40)
        
        # Auto publish
        auto_publish = input("Auto-publish clips immediately? (y/n) [y]: ").strip().lower()
        self.config.config['upload_settings']['auto_publish'] = auto_publish not in ['n', 'no', 'false']
        
        # Stagger uploads
        stagger = input("Stagger uploads over time? (y/n) [y]: ").strip().lower()
        if stagger in ['n', 'no', 'false']:
            self.config.config['upload_settings']['stagger_uploads'] = False
        else:
            self.config.config['upload_settings']['stagger_uploads'] = True
            interval = input("Minutes between uploads (5-120) [30]: ").strip()
            if interval and interval.isdigit() and 5 <= int(interval) <= 120:
                self.config.config['upload_settings']['stagger_minutes'] = int(interval)
    
    def _setup_caption_settings(self):
        """Configure caption appearance"""
        print("\n" + "-"*40)
        print("STEP 5: CAPTION SETTINGS")
        print("-"*40)
        
        # Font size
        font_size = input("Caption font size (12-48) [24]: ").strip()
        if font_size and font_size.isdigit() and 12 <= int(font_size) <= 48:
            self.config.config['caption_settings']['font_size'] = int(font_size)
        
        # Font color
        print("Available colors: white, yellow, red, blue, green, black")
        font_color = input("Caption font color [white]: ").strip().lower()
        if font_color in ['white', 'yellow', 'red', 'blue', 'green', 'black']:
            self.config.config['caption_settings']['font_color'] = font_color

class VideoValidator:
    """Validate video files before processing"""
    
    @staticmethod
    def validate_video(video_path: str) -> Tuple[bool, str]:
        """Validate if video file is suitable for processing"""
        if not os.path.exists(video_path):
            return False, "File does not exist"
        
        try:
            # Check file size (minimum 1MB, maximum 2GB)
            file_size = os.path.getsize(video_path)
            if file_size < 1024 * 1024:  # 1MB
                return False, "Video file too small (minimum 1MB)"
            if file_size > 2 * 1024 * 1024 * 1024:  # 2GB
                return False, "Video file too large (maximum 2GB)"
            
            # Check if ffprobe can read the file
            cmd = ["ffprobe", "-v", "quiet", "-select_streams", "v:0", 
                   "-show_entries", "stream=duration", "-of", "csv=p=0", video_path]
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode != 0:
                return False, "Invalid video format"
            
            duration = float(result.stdout.strip())
            if duration < 30:  # Minimum 30 seconds
                return False, "Video too short (minimum 30 seconds)"
            if duration > 3600:  # Maximum 1 hour
                return False, "Video too long (maximum 1 hour)"
            
            return True, "Valid video file"
            
        except Exception as e:
            return False, f"Validation error: {str(e)}"

class ConfigValidator:
    """Validate and check configuration completeness"""
    
    @staticmethod
    def validate_setup(config: ConfigManager) -> Tuple[bool, List[str]]:
        """Check if configuration is complete and valid"""
        issues = []
        
        # Check platforms
        platforms = config.get('upload_settings.platforms', [])
        if not platforms:
            issues.append("No upload platforms configured")
        
        # Check API keys for selected platforms
        for platform in platforms:
            if platform == 'youtube':
                if not config.get('api_keys.youtube_client_id'):
                    issues.append("YouTube Client ID missing")
                if not config.get('api_keys.youtube_client_secret'):
                    issues.append("YouTube Client Secret missing")
            elif platform == 'tiktok':
                if not config.get('api_keys.tiktok_client_key'):
                    issues.append("TikTok Client Key missing")
            elif platform == 'instagram':
                if not config.get('api_keys.instagram_access_token'):
                    issues.append("Instagram Access Token missing")
            elif platform == 'linkedin':
                if not config.get('api_keys.linkedin_client_id'):
                    issues.append("LinkedIn Client ID missing")
        
        return len(issues) == 0, issues
    
    @staticmethod
    def show_configuration_status(config: ConfigManager):
        """Display current configuration status"""
        print("\n" + "="*50)
        print("         CONFIGURATION STATUS")
        print("="*50)
        
        # Video settings
        print("\nVIDEO SETTINGS:")
        print(f"  Clip Duration: {config.get('video_settings.clip_duration', 12)} seconds")
        print(f"  Max Clips: {config.get('video_settings.max_clips', 20)}")
        print(f"  Video Quality: {config.get('video_settings.video_quality', 18)} CRF")
        
        # Platforms
        platforms = config.get('upload_settings.platforms', [])
        print(f"\nUPLOAD PLATFORMS: {', '.join(platforms) if platforms else 'None'}")
        
        # Upload settings
        print("\nUPLOAD SETTINGS:")
        print(f"  Auto-publish: {config.get('upload_settings.auto_publish', True)}")
        print(f"  Stagger uploads: {config.get('upload_settings.stagger_uploads', True)}")

def check_dependencies() -> List[str]:
    """Check if all required dependencies are available"""
    missing = []
    
    # Check FFmpeg
    try:
        subprocess.run(['ffmpeg', '-version'], capture_output=True, check=True)
    except (subprocess.CalledProcessError, FileNotFoundError):
        missing.append("ffmpeg")
    
    # Check ffprobe
    try:
        subprocess.run(['ffprobe', '-version'], capture_output=True, check=True)
    except (subprocess.CalledProcessError, FileNotFoundError):
        missing.append("ffprobe")
    
    # Check Python packages
    required_packages = ['whisper', 'requests']
    for package in required_packages:
        try:
            __import__(package)
        except ImportError:
            missing.append(f"python package: {package}")
    
    return missing

def setup_directories():
    """Create necessary directories"""
    directories = ['clips', 'temp_clips', 'logs']
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

def is_notebook_environment():
    """Check if running in Jupyter notebook"""
    try:
        get_ipython
        return True
    except NameError:
        return False

def main():
    """Main entry point for the application"""
    
    # Check if running in notebook environment
    if is_notebook_environment():
        # If in notebook, provide interactive interface
        print("Video Clipper detected Jupyter environment")
        print("Use the VideoClipperApp class directly:")
        print("app = VideoClipperApp()")
        print("result = app.process_video('ssvid.net--Journey-Through-Switzerland-2-Minutes-Cinematic-Travel-Video_v720P.mp4')")
        return
    
    import argparse
    
    parser = argparse.ArgumentParser(description='AI Video Clipper and Multi-Platform Uploader')
    parser.add_argument('video_path', nargs='?', help='Path to the input video file')
    parser.add_argument('--output-dir', default='clips', help='Output directory for clips')
    parser.add_argument('--config', default='config.json', help='Configuration file path')
    parser.add_argument('--batch', action='store_true', help='Process multiple videos')
    parser.add_argument('--analytics', action='store_true', help='Show analytics report')
    parser.add_argument('--setup', action='store_true', help='Run setup wizard')
    parser.add_argument('--status', action='store_true', help='Show configuration status')
    
    # Filter out Jupyter kernel arguments
    filtered_args = []
    skip_next = False
    
    for i, arg in enumerate(sys.argv[1:], 1):
        if skip_next:
            skip_next = False
            continue
        if arg in ['-f', '--kernel']:
            skip_next = True
            continue
        if arg.startswith('--kernel=') or arg.startswith('-f='):
            continue
        filtered_args.append(arg)
    
    args = parser.parse_args(filtered_args)
    
    # Initialize configuration
    config = ConfigManager(args.config)
    
    # Handle setup wizard
    if args.setup:
        wizard = SetupWizard(config)
        wizard.run_setup()
        return
    
    # Handle status check
    if args.status:
        ConfigValidator.show_configuration_status(config)
        return
    
    # Check dependencies
    missing_deps = check_dependencies()
    if missing_deps:
        print("Missing dependencies:")
        for dep in missing_deps:
            print(f"  - {dep}")
        print("\nPlease install missing dependencies before running.")
        return
    
    # Setup directories
    setup_directories()
    
    # Initialize app
    app = VideoClipperApp(args.config)
    
    if args.analytics:
        report = app.get_analytics_report()
        print("\nANALYTICS REPORT")
        print("="*50)
        print(f"Total clips generated: {report['summary']['total_clips']}")
        print(f"Total uploads completed: {report['summary']['total_uploads']}")
        return
    
    # Require video path for processing
    if not args.video_path:
        print("\nUsage: python video_clipper.py <video_path> [options]")
        print("Run --setup for initial configuration")
        return
    
    if args.batch:
        # Batch processing
        if not os.path.isdir(args.video_path):
            print(f"Error: {args.video_path} is not a directory")
            return
        
        video_extensions = ['.mp4', '.mov', '.avi', '.mkv']
        video_files = []
        for ext in video_extensions:
            video_files.extend(Path(args.video_path).glob(f'*{ext}'))
        
        if not video_files:
            print(f"No video files found in {args.video_path}")
            return
        
        results = app.batch_process_videos([str(vf) for vf in video_files])
        print(f"Batch completed: {len(results['successful'])} successful, {len(results['failed'])} failed")
    else:
        # Single video processing
        if not os.path.exists(args.video_path):
            print(f"Error: Video file {args.video_path} not found")
            return
        
        # Validate video
        is_valid, message = VideoValidator.validate_video(args.video_path)
        if not is_valid:
            print(f"Error: {message}")
            return
        
        print(f"Processing video: {args.video_path}")
        result = app.process_video(args.video_path, args.output_dir)
        
        if result['status'] == 'success':
            print(f"Success! Generated {result['clips_generated']} clips")
            print(f"Output directory: {args.output_dir}")
        else:
            print(f"Error: {result['error']}")

# For Jupyter notebook usage
def create_app(config_file: str = "config.json") -> VideoClipperApp:
    """Convenience function to create app instance in notebooks"""
    return VideoClipperApp(config_file)

if __name__ == "__main__":
    main()

Video Clipper detected Jupyter environment
Use the VideoClipperApp class directly:
app = VideoClipperApp()
result = app.process_video('ssvid.net--Journey-Through-Switzerland-2-Minutes-Cinematic-Travel-Video_v720P.mp4')


In [23]:
app = VideoClipperApp()

2025-08-28 16:39:54,403 - INFO - Video Clipper App initialized


In [24]:
config = ConfigManager()
ConfigValidator.show_configuration_status(config)


         CONFIGURATION STATUS

VIDEO SETTINGS:
  Clip Duration: 12 seconds
  Max Clips: 20
  Video Quality: 18 CRF

UPLOAD PLATFORMS: youtube, tiktok, instagram, linkedin

UPLOAD SETTINGS:
  Auto-publish: True
  Stagger uploads: True


In [25]:
result = app.process_video('ssvid.net--Journey-Through-Switzerland-2-Minutes-Cinematic-Travel-Video_v720P.mp4')

2025-08-28 16:40:07,054 - INFO - Starting video processing for ssvid.net--Journey-Through-Switzerland-2-Minutes-Cinematic-Travel-Video_v720P.mp4
2025-08-28 16:40:07,055 - INFO - Step 1: Generating clips
2025-08-28 16:40:07,056 - INFO - Starting clip generation from ssvid.net--Journey-Through-Switzerland-2-Minutes-Cinematic-Travel-Video_v720P.mp4
2025-08-28 16:40:15,768 - INFO - Created clip 1: clips/clip_000_0.mp4
2025-08-28 16:40:28,481 - INFO - Created clip 2: clips/clip_001_10.mp4
2025-08-28 16:40:42,296 - INFO - Created clip 3: clips/clip_002_20.mp4
2025-08-28 16:40:57,489 - INFO - Created clip 4: clips/clip_003_30.mp4
2025-08-28 16:41:11,869 - INFO - Created clip 5: clips/clip_004_40.mp4
2025-08-28 16:41:29,809 - INFO - Created clip 6: clips/clip_005_50.mp4
2025-08-28 16:41:48,821 - INFO - Created clip 7: clips/clip_006_60.mp4
2025-08-28 16:42:10,078 - INFO - Created clip 8: clips/clip_007_70.mp4
2025-08-28 16:42:30,035 - INFO - Created clip 9: clips/clip_008_80.mp4
2025-08-28 16: